<div>
    <img src="https://storage.googleapis.com/kaggle-competitions/kaggle/23652/logos/header.png" />
</div>

In [ ]:
import numpy as np
import pandas as pd

import os
import sys

effnet_base_path = '../input/efficientnet-pytorch/EfficientNet-PyTorch/EfficientNet-PyTorch-master'

sys.path = [effnet_base_path,] + sys.path

from tqdm import tqdm
import torch
import torch.nn as nn
from efficientnet_pytorch import model as enet

<h1 id="dataset" style="color:#fefd1c; background:#0e1b1f; border:1px dotted #fefd1c;"> 
    <center>Dataset
        <a class="anchor-link" href="#dataset" target="_self">¶</a>
    </center>
</h1>

In [ ]:
class ClassificationDataset:
    
    def __init__(self, image_paths, targets): 
        self.image_paths = image_paths
        self.targets = targets

    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, item):      
        image = np.load(self.image_paths[item]).astype(float)

        targets = self.targets[item]
                
        return {
            "image": torch.tensor(image, dtype=torch.float),
            "targets": torch.tensor(targets, dtype=torch.long),
        }

In [ ]:
df = pd.read_csv('../input/seti-breakthrough-listen/train_labels.csv')
df['img_path'] = df['id'].apply(lambda x: f'../input/seti-breakthrough-listen/train/{x[0]}/{x}.npy')

<h1 id="model" style="color:#fefd1c; background:#0e1b1f; border:1px dotted #fefd1c;"> 
    <center>Model
        <a class="anchor-link" href="#model" target="_self">¶</a>
    </center>
</h1>

In [ ]:
class enetv2(nn.Module):
    def __init__(self, backbone, out_dim):
        super(enetv2, self).__init__()
        self.enet = enet.EfficientNet.from_name(backbone)
        self.enet.load_state_dict(torch.load(pretrained_models[backbone]))
        self.myfc = nn.Linear(self.enet._fc.in_features, out_dim)
        self.enet._fc = nn.Identity()
        self.conv1 = nn.Conv2d(6, 3, kernel_size=3, stride=1, padding=3, bias=False)

    def extract(self, x):
        return self.enet(x)

    def forward(self, x):
        x = self.conv1(x)
        x = self.extract(x)
        x = self.myfc(x)
        return x

<h1 id="load" style="color:#fefd1c; background:#0e1b1f; border:1px dotted #fefd1c;"> 
    <center>Load Models
        <a class="anchor-link" href="#load" target="_self">¶</a>
    </center>
</h1>

## Weights

In [ ]:
pretrained_models = {
    'efficientnet-b0': '../input/efficientnet-pytorch/efficientnet-b0-08094119.pth',
    'efficientnet-b1': '../input/efficientnet-pytorch/efficientnet-b1-dbc7070a.pth',
    'efficientnet-b2': '../input/efficientnet-pytorch/efficientnet-b2-27687264.pth',
    'efficientnet-b3': '../input/efficientnet-pytorch/efficientnet-b3-c8376fa2.pth',
    'efficientnet-b4': '../input/efficientnet-pytorch/efficientnet-b4-e116e8b3.pth',
    'efficientnet-b5': '../input/efficientnet-pytorch/efficientnet-b5-586e6cc6.pth',
    'efficientnet-b6': '../input/efficientnet-pytorch/efficientnet-b6-c76e70fd.pth',
}

pretrained_weights = {
    'efficientnet-b0': '../input/seti-effnet/efficientnet-b0-4.pt',
    'efficientnet-b1': '../input/seti-effnet/efficientnet-b1-4.pt',
    'efficientnet-b2': '../input/seti-effnet/efficientnet-b2-4.pt',
    'efficientnet-b3': '../input/seti-effnet/efficientnet-b3-4.pt',
    'efficientnet-b4': '../input/seti-effnet/efficientnet-b4-4.pt',
    'efficientnet-b5': '../input/seti-effnet/efficientnet-b5-4.pt',
    'efficientnet-b6': '../input/seti-effnet/efficientnet-b6-4.pt',
}

## Load Models

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

models = []
for key in pretrained_models.keys():
    model = enetv2(key, out_dim=1)
    model.load_state_dict(torch.load(pretrained_weights[key], map_location=torch.device(device)))
    models.append(model.to(device))

<h1 id="predict" style="color:#fefd1c; background:#0e1b1f; border:1px dotted #fefd1c;"> 
    <center>Predictions
        <a class="anchor-link" href="#predict" target="_self">¶</a>
    </center>
</h1>

In [ ]:
def predict(data_loader, model, device):
    model.eval()
    
    final_targets = []
    final_outputs = []
    
    with torch.no_grad():
        
        for data in tqdm(data_loader, position=0, leave=True, desc='Evaluating'):
            inputs = data["image"]
            targets = data["targets"]
            inputs = inputs.to(device, dtype=torch.float)
            targets = targets.to(device, dtype=torch.float)
            
            output = model(inputs)
            
            targets = targets.detach().cpu().numpy().tolist()
            output = output.detach().cpu().numpy().tolist()
            
            final_targets.extend(targets)
            final_outputs.extend(output)
            
    return final_outputs, final_targets

## Prepare submission dataset

In [ ]:
submission = pd.read_csv('../input/seti-breakthrough-listen/sample_submission.csv')
submission['img_path'] = submission['id'].apply(lambda x: f'../input/seti-breakthrough-listen/test/{x[0]}/{x}.npy')

test_dataset = ClassificationDataset(image_paths=submission.img_path.values, targets=submission.target.values)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=16, shuffle=False, num_workers=4)

## Predictions for each model

In [ ]:
sig = torch.nn.Sigmoid()
outs = []
for model in models:
    predictions, valid_targets = predict(test_loader, model, device=device)
    predictions = np.array(predictions)[:, 0]
    out = sig(torch.from_numpy(predictions))
    out = out.detach().numpy()
    outs.append(out)

In [ ]:
# take the mean of all predictions
pred = np.mean(np.array(outs), axis=0)

submission.target = pred
submission.drop(['img_path'], axis=1, inplace=True)
submission.to_csv('submission.csv', index=False)

<h1 id="references" style="color:#fefd1c; background:#0e1b1f; border:1px dotted #fefd1c;"> 
    <center>References
        <a class="anchor-link" href="#references" target="_self">¶</a>
    </center>
</h1>

[Byfone training notebook](https://www.kaggle.com/byfone/efficientnetb1-on-seti-breakthrough-listen)